In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers
from keras.callbacks import CSVLogger
#from livelossplot import PlotLossesKeras
import os
import numpy as np
#from imgaug import augmenters as iaa
#import cv2
from keras.layers.normalization import BatchNormalization
import seaborn as sns
import pandas as pd 
from keras import initializers
from keras import optimizers
import keras.backend as K
import tensorflow as tf
from keras.optimizers import SGD

import numpy as np
import os
import glob
import cv2
import random
import matplotlib.pyplot as plt

#import keras_metrics
from imgaug import augmenters as iaa
import tensorflow as tf
#from sklearn.cross_validation import StratifiedKFold
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras.utils import to_categorical, plot_model
from keras.layers import Dense, Input, Conv2D, Flatten, MaxPooling2D, Activation
from keras.models import Model, Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
from keras.models import load_model
from keras.utils import np_utils
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
np.random.seed(1)

Using TensorFlow backend.


In [2]:
x_all = np.load("datasets/all/x_all_200.npy")
y_all = np.load("datasets/all/y_all_200.npy")
x_all.shape, y_all.shape

((27558, 200, 200, 3), (27558,))

In [3]:
# one hot encoded vectors
num_classes = 2

y_all = np_utils.to_categorical(y_all,num_classes)
y_all.shape

(27558, 2)

In [4]:
IMAGE_SIZE = x_all.shape[1]

def get_model_exp2_b():
    base_model = VGG16(weights='imagenet',include_top=False,pooling='avg',input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    base_model.trainable = False
    X = base_model.output
    X.trainable = False
    
    X = Dense(1024, activation='relu')(X)
    X = Dropout(0.5)(X)
    predictions = Dense(2, activation='softmax', trainable=True)(X)
    
    for layer in base_model.layers[-4:]:
        layer.trainable=False
    
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = None
model=get_model_exp2_b()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0         
__________

In [5]:
def train_model():
    #model = None
    #model=get_model_exp2_b()
    #model.summary()
    
    x_train, x_val, y_train, y_val = train_test_split(x_all, y_all, test_size=0.10, random_state=42)
    
    from keras.preprocessing.image import ImageDataGenerator

    model=get_model_exp2_b()
    path_model='vgg_sterioid.h5' 


    # set the learning rate
    K.set_value(model.optimizer.lr, 0.01) 


    datagen = ImageDataGenerator(
        featurewise_center=False,
        featurewise_std_normalization=False,
        rotation_range=35,
        width_shift_range=0.25,
        height_shift_range=0.25,
        shear_range = 20,
        horizontal_flip=True)

    datagen.fit(x_train)

    model.fit_generator(datagen.flow(x_train, y_train, batch_size=64),
                        steps_per_epoch=len(x_train) / 64, 
                        epochs=100, 
                        verbose=1, 
                        validation_data=(x_val,y_val),
                        callbacks=[
                           ModelCheckpoint(filepath=path_model, monitor='val_acc',  save_best_only=False),
                        ]      
                       )
    return history
    